In [ ]:
#!pip install pydrive

In [1]:
import os

In [4]:
contenido = os.listdir(r'D:\PYTHON\utea_scripts\reportes_credito_siembra\pdfs')

dic_files = {}
for file in contenido:
    split = file.split('_')
    dic_files[int(split[0])] = file
dic_files

{13132: '13132_ZUMARAN MENACHO JORGE.pdf',
 18088: '18088_FRERKING ORTIZ RICARDO.pdf',
 2816: '2816_DONOSO BLANCO ALBERTO.pdf',
 42154: '42154_PIZARRO CABRERA ARMANDO CESAR.pdf',
 42191: '42191_CALVI LEON MARIOLY.pdf',
 515: '515_AGUILERA OLGA RIVERO DE.pdf',
 594: '594_AGROPECUARIA FLAMBOYAN.pdf',
 7506: '7506_LOHENER ROSENDA SHEILA PAZ DE.pdf',
 86: '86_AGUILERA TARADELLES JOSE LUIS.pdf'}

#  CARGAR FILES A DRIVE

In [5]:
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive

gauth = GoogleAuth()
gauth.LocalWebserverAuth() # client_secrets.json need to be in the same directory as the script
drive = GoogleDrive(gauth)

Your browser has been opened to visit:

    https://accounts.google.com/o/oauth2/auth?client_id=1096764895697-57btfmtn7rdejufct5qq4j9mi61j0unl.apps.googleusercontent.com&redirect_uri=http%3A%2F%2Flocalhost%3A8080%2F&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive&access_type=offline&response_type=code

Authentication successful.


In [6]:
print(len(dic_files), 'INFORMES')
dic_files

9 INFORMES


{13132: '13132_ZUMARAN MENACHO JORGE.pdf',
 18088: '18088_FRERKING ORTIZ RICARDO.pdf',
 2816: '2816_DONOSO BLANCO ALBERTO.pdf',
 42154: '42154_PIZARRO CABRERA ARMANDO CESAR.pdf',
 42191: '42191_CALVI LEON MARIOLY.pdf',
 515: '515_AGUILERA OLGA RIVERO DE.pdf',
 594: '594_AGROPECUARIA FLAMBOYAN.pdf',
 7506: '7506_LOHENER ROSENDA SHEILA PAZ DE.pdf',
 86: '86_AGUILERA TARADELLES JOSE LUIS.pdf'}

In [9]:
dic_urls = {}
# View all folders and file in your Google Drive
fileList = drive.ListFile({'q': "'root' in parents and trashed=false"}).GetList()
for file in fileList:
  #print('Title: %s, ID: %s' % (file['title'], file['id']))
  # Get the folder ID that you want
  if(file['title'] == "1RA_INSPECCION"):
      fileID = file['id']

for file in dic_files:
    file1 = drive.CreateFile({"mimeType": "application/pdf", "parents": [{"kind": "drive#fileLink", "id": fileID}]})
    file1['title'] = dic_files[file]
    file1.SetContentFile(r'D:\PYTHON\utea_scripts\reportes_credito_siembra\pdfs//' + dic_files[file])
    file1.Upload() # Upload the file.
    print('Created file %s with mimeType %s' % (file1['title'], file1['mimeType']))
    permission = file1.InsertPermission({
                        'type': 'anyone',
                        'value': 'anyone',
                        'role': 'reader'})
    dic_urls[file] = file1['alternateLink']
dic_urls
print('end')

Created file 13132_ZUMARAN MENACHO JORGE.pdf with mimeType application/pdf
Created file 18088_FRERKING ORTIZ RICARDO.pdf with mimeType application/pdf
Created file 2816_DONOSO BLANCO ALBERTO.pdf with mimeType application/pdf
Created file 42154_PIZARRO CABRERA ARMANDO CESAR.pdf with mimeType application/pdf
Created file 42191_CALVI LEON MARIOLY.pdf with mimeType application/pdf
Created file 515_AGUILERA OLGA RIVERO DE.pdf with mimeType application/pdf
Created file 594_AGROPECUARIA FLAMBOYAN.pdf with mimeType application/pdf
Created file 7506_LOHENER ROSENDA SHEILA PAZ DE.pdf with mimeType application/pdf
Created file 86_AGUILERA TARADELLES JOSE LUIS.pdf with mimeType application/pdf
end


In [10]:
print(len(dic_urls), 'URLs')
dic_urls

9 URLs


{13132: 'https://drive.google.com/file/d/1xlejwzRx3Dq6iBzeLisrsLKE7eqddRrI/view?usp=drivesdk',
 18088: 'https://drive.google.com/file/d/1ArH4lMotAytLVvY7EamVuijfucWExQ_r/view?usp=drivesdk',
 2816: 'https://drive.google.com/file/d/1uCaA5ecEHmgTukwk4MPDd0T961_GtVg_/view?usp=drivesdk',
 42154: 'https://drive.google.com/file/d/1e6fXf-whgzuujFoQVNAObM1EBDg6c6S8/view?usp=drivesdk',
 42191: 'https://drive.google.com/file/d/1guW8dCLiiG7KoMlaQVZfE1GRlvz_PFl4/view?usp=drivesdk',
 515: 'https://drive.google.com/file/d/11Wh31FjxRBd4IGb4fDZR8Go3zJPIdWpC/view?usp=drivesdk',
 594: 'https://drive.google.com/file/d/1Y2XYf01__kRFXJCjK4tRwrsSsNbWln00/view?usp=drivesdk',
 7506: 'https://drive.google.com/file/d/16skeJXdRBQMNb8WGS3OXe5o_cANdAV6m/view?usp=drivesdk',
 86: 'https://drive.google.com/file/d/1PfWAityoc55ZNPFRqIY9T-neE-7Omg5J/view?usp=drivesdk'}

# CONECTAR A AMIGOCLOUD

In [12]:
from amigocloud import AmigoCloud

In [15]:
# CONECTAR A AMIGOCLOUD
acceso = open('../../../api_key/api.key','r')
api_token = acceso.readlines()[0]
acceso.close()
amigocloud = AmigoCloud(token=api_token)
amigocloud

In [16]:
for key in dic_urls:
    query = {'query': 'update dataset_307560 set link_doc_1ra = \'{link}\' where cast(SPLIT_PART(canhero, \' / \', 1) as integer) = \'{cod}\''.format(link=dic_urls[key], cod=key)}
    res = amigocloud.post('https://app.amigocloud.com/api/v1/projects/32672/sql', query)
    print(key)
print('end')

13132
18088
2816
42154
42191
515
594
7506
86
end


# -----------------------------------------------------------

# CONECCION A DRIVE

In [ ]:
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive

gauth = GoogleAuth()
gauth.LocalWebserverAuth() # client_secrets.json need to be in the same directory as the script
drive = GoogleDrive(gauth)

In [ ]:
# View all folders and file in your Google Drive
fileList = drive.ListFile({'q': "'root' in parents and trashed=false"}).GetList()
for file in fileList:
  print('Title: %s, ID: %s' % (file['title'], file['id']))
  # Get the folder ID that you want
  if(file['title'] == "REPORTES_CREDITO"):
      fileID = file['id']

file1 = drive.CreateFile({"mimeType": "application/pdf", "parents": [{"kind": "drive#fileLink", "id": fileID}]})
file1.SetContentFile("file.pdf")
file1.Upload() # Upload the file.
print('Created file %s with mimeType %s' % (file1['title'], file1['mimeType']))

In [ ]:
permission = file1.InsertPermission({
                        'type': 'anyone',
                        'value': 'anyone',
                        'role': 'reader'})

print(file1['alternateLink'])  # Display the sharable link.

In [ ]:
for file in fileList[0]:
    print(file, '       ',fileList[0][file])

In [ ]:
fileList[0]['title']